In [116]:
from opensky_api import OpenSkyApi
from kafka import KafkaProducer
import json
from time import sleep
import pyarrow as pa
import os
from datetime import datetime
import shutil
    
# # Topics/Brokers
topic_real_time_states = 'real-time-states'
topic_raw_json = 'raw_json'
topic_flat_json = 'flat_json'
topic_brokers = ['localhost:9092']

# hdfs config
hdfs_host='localhost'
hdfs_port=8020
hdfs_user='hdfs'
hdfs_driver='libhdfs'
hdfs_archive_path = '/FinalProject/Archive'
local_files_folder = os.path.abspath('/home/naya/tutorial/open-sky/tmp_storage')
local_files_prefix = 'opensky_state_vectors_json'
local_files_suffix = lambda : datetime.now().strftime("%Y-%M-%d_%H_%M_%S")
local_file_size_threshold = 100 #MB

API = OpenSkyApi('livgeni', '1abc23')

def open_sky_generator():
    while True:
        yield API.get_states()
        sleep(10)


In [103]:
import json

RED_FIELDS = ['origin_country', 'sensors']
YELLOW_FIELDS = ['time_position', 'heading', 'vertical_rate', 'spi']
IGNORE_FIELDS = RED_FIELDS + YELLOW_FIELDS


def opensky_to_dict(opensky_states):
    """returns a list of dictionaries"""
    svdl = list()

    for sv in opensky_states.states:
        svd = dict(time = opensky_states.time)
        for key in sv.keys:
            if key not in IGNORE_FIELDS:
                val = sv.__dict__[key]
                svd[key] = val if type(val) != str else val.strip()
        svdl += [svd]
    return svdl
        
#     state_vector_json_list = json.dumps(svdl)
#     return state_vector_json_list



## Save to Archiving Folder in HDFS

1. Append to a local file until reaching 100MB
2. When reached - upload to HDFS using pyarrow and open a new file

In [117]:
# init local and hdfs folders

fs = pa.hdfs.connect(
    host=hdfs_host, 
    port=hdfs_port, 
    user=hdfs_user, 
    kerb_ticket=None, 
    driver=hdfs_driver, 
    extra_conf=None)

# Create local folder if not exists
if os.path.exists(local_files_folder):
    shutil.rmtree(local_files_folder)


os.mkdir(local_files_folder)

# create hdfs folder if not exist
if not fs.exists(hdfs_archive_path):
    fs.mkdir(f'hdfs://{hdfs_host}:{hdfs_port}{hdfs_archive_path}',create_parents=True)

local_file_path = os.path.join\
        (local_files_folder, f'{local_files_prefix}_{local_files_suffix()}.json')
    
def append_data(jsn_str):
    with open(local_file_path, 'a') as local_file:
        local_file.write(jsn_str)
    
    if os.path.getsize(local_file_path)/1024/1024 >= local_file_size_threshold:
        # upload to hdfs
        
        # delete the local file
        os.remove(local_file_path)
    
    

In [118]:
producer = KafkaProducer(bootstrap_servers=brokers)

for states in open_sky_generator():
    message_json = json.dumps(opensky_to_dict(states))
    append_data(message_json)
#    producer.send(topic_raw_json, value = message_json[:900000].encode('utf-8'))
#     producer.send(topic_raw_json, b'message')
    print(len(message_json.encode()))
    print(len(message_json.encode('utf-8'))/1024/1024)

file size: 2166770
2166770
2.0663928985595703


KeyboardInterrupt: 

In [87]:
local_file_path = os.path.join(local_files_folder, local_files_prefix+'_'+local_files_suffix())
local_file_path

'/home/naya/tutorial/open-sky/tmp_storage/opensky_state_vectors_json_2019-08-12_21_08_48'

In [101]:
local_file_path = os.path.join\
(local_files_folder, f'{local_files_prefix}_{local_files_suffix()}')